Importações

In [ ]:
import numpy as np
import pandas as pd
import folium
import geopandas as gpd
from sqlalchemy import create_engine  
import getpass
from folium import plugins

Conexão com o banco de dados do Postgres

In [ ]:
host = ""
database = ""
user = ''
password = ''
# password = getpass.getpass()
connection_string = f"postgresql+psycopg2://{user}:{password}@{host}/{database}"

Mapa dos Biomas

In [ ]:
sql = "select * from biomas_br"
df_biomas = gpd.GeoDataFrame.from_postgis(sql,connection_string,geom_col='geom')
df_biomas.head()

In [ ]:
mapa1 = folium.Map([-14.0988176, -56.2574336], zoom_start=4)

gjson_biomas = df_biomas.__geo_interface__
bioma_color = {
    'Amazônia': 'purple',
    'Caatinga': 'blue',
    'Cerrado': 'red',
    'Mata Atlântica': 'yellow',
    'Pampa': 'pink',
    'Pantanal': 'green',
}

style = lambda feature: {
    'fillColor': bioma_color[feature['properties']['bioma']],
    'color': 'black',
    'fillOpacity': 0.5,
    'weight': 1.0,
}

mapa_1=folium.GeoJson(gjson_biomas, name='Biomas do Brasil',style_function=style).add_to(mapa1)

mapa_1.add_to(mapa1)

mapa1.save('biomas.html')
mapa1

In [ ]:
'''
#mapa com descrição do bioma
perso1 = folium.features.GeoJson(
    data = gjson_biomas,
    style_function=style,  
    control=False, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['bioma'],
        aliases=['Bioma:']
    )
)

mapa1.add_child(perso1)
mapa1.keep_in_front(perso1)
mapa_1=folium.GeoJson(gjson_biomas, name='Biomas do Brasil',style_function=style)
mapa_1.add_to(mapa1)
mapa1
'''

Mapa do Desmatamento no Bioma Amazônia

In [ ]:
sql = "select * from desm_amaz"
df_desmatamento = gpd.GeoDataFrame.from_postgis(sql,connection_string,geom_col='geom')
df_desmatamento.head()

In [ ]:
df_amazo_desmat = df_desmatamento[df_desmatamento['nome'].isin(df_desmatamento['nome'])]

mapa2 = folium.Map([-5.4888176, -59.8374336], zoom_start=5)

gjson_desmatamento = df_amazo_desmat.__geo_interface__

mapa_2=folium.Choropleth(
    geo_data=gjson_desmatamento,
    name='Desmatamento no Bioma Amazônia',
    data=df_amazo_desmat,
    columns=['nome', 'a_desma'],
    key_on='feature.properties.nome',
    fill_color='OrRd',
    fill_opacity=0.8,
    line_opacity=0.2,
    line_color='black',
    legend_name='Desmatamento (Km²)',
    highlight=True
    
)
mapa_2.add_to(mapa2)
mapa2.save('desmatamento.html')
mapa2

In [ ]:
'''
#mapa com descrição da quantidade
style_2 = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}

highlight_2 = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
perso = folium.features.GeoJson(
    data = gjson_desmatamento,
    style_function=style_2,  
    control=False,
    highlight_function=highlight_2, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['a_desma'],
        aliases=['Destamatamento(km²):'],
    )
)

mapa2.add_child(perso)
mapa2.keep_in_front(perso)
mapa2
'''

Mapa das Áreas Indígenas

In [ ]:
sql = "select * from a_indigenas"
df_indigenas = gpd.GeoDataFrame.from_postgis(sql,connection_string,geom_col='geom')
df_indigenas.head()

In [ ]:
mapa3 = folium.Map([-14.0988176, -56.2574336], zoom_start=4)

df_indigenas['Latitude'] = df_indigenas['geom'].y.astype(float)
df_indigenas['Longitude'] = df_indigenas['geom'].x.astype(float)

mapa_3 = plugins.MarkerCluster(name='Áreas indígenas nos Biomas').add_to(mapa3)

for i, row in df_indigenas.iterrows():
    location = [row['Latitude'], row['Longitude']]
    nome = row['nome']
    marca= folium.Marker(location=location, popup=nome)
    marca.add_to(mapa_3)

mapa_3.add_to(mapa3)

mapa3.save('area_indigenas.html')
mapa3

Mapa das Áreas Indígenas no Bioma Amazônia

In [ ]:
sql = "select a.* from a_indigenas a, biomas_br b where ST_Intersects(a.geom, b.geom) and b.bioma='Amazônia'"
df_zonaindigena = gpd.GeoDataFrame.from_postgis(sql,connection_string,geom_col='geom')
df_zonaindigena.head()

In [ ]:
mapa4 = folium.Map([-7.6188176, -56.8374336],zoom_start = 5)

mapa_4 = folium.FeatureGroup(name='Áreas indígenas no Bioma Amazônia')
df_zonaindigena.apply(lambda row: folium.Marker(location = [row.loc['geom'].y, row.loc['geom'].x],
                                         popup=row.loc['nome'],icon = folium.Icon(color='green')).add_to(mapa_4), axis=1)

mapa_4.add_to(mapa4)
mapa4.save("indigenas_amazonia.html")
mapa4

Legenda

In [ ]:
legenda = """
    <div style='position: fixed; bottom: 50px; right: 50px; z-index: 1000; background-color: white; padding: 10px; border-radius: 5px;'>
        <h4>Biomas</h4>
        <div>
            <span style='background-color: purple; opacity:0.7; width: 10px; height: 10px; display: inline-block; margin-right: 5px;'></span>
            Amazônia
        </div>
        <div>
            <span style='background-color: blue; opacity:0.7; width: 10px; height: 10px; display: inline-block; margin-right: 5px;'></span>
            Caatinga
        </div>
        <div>
            <span style='background-color: red; opacity:0.7; width: 10px; height: 10px; display: inline-block; margin-right: 5px;'></span>
            Cerrado
        </div>
        <div>
            <span style='background-color: yellow; opacity:0.7; width: 10px; height: 10px; display: inline-block; margin-right: 5px;'></span>
            Mata Atlântica
        </div>
        <div>
            <span style='background-color: pink; opacity:0.7; width: 10px; height: 10px; display: inline-block; margin-right: 5px;'></span>
            Pampa
        </div>
        <div>
            <span style='background-color: green; opacity:0.7; width: 10px; height: 10px; display: inline-block; margin-right: 5px;'></span>
            Pantanal
        </div>
        <h4>Áreas</h4>
        <div>
            <span class="fa fa-map-marker fa-lg" style='color: blue; opacity:0.7; width: 10px; height: 10px; display: inline-block; margin-right: 5px;'></span>
            Áreas indígenas
        </div><br>
        <div>
            <span class="fa fa-map-marker fa-lg" style='color: green; opacity:0.7; width: 10px; height: 10px; display: inline-block; margin-right: 5px;'></span>
            Áreas indígenas da Amazônia
        </div>
    </div>

"""

mapa1.get_root().html.add_child(folium.Element(legenda))
mapa_1.add_to(mapa1)


Camadas Customizadas

In [ ]:
mapa_base = folium.Map([-14.0988176, -56.2574336], zoom_start=4)

#camada1
mapa_base.get_root().html.add_child(folium.Element(legenda))
mapa_1.add_to(mapa_base)
#camada2
mapa_2.add_to(mapa_base)
#camada3
mapa_3.add_to(mapa_base)
#camada4
mapa_4.add_to(mapa_base)                                      
    
folium.LayerControl().add_to(mapa_base)

mapa_base.save('camadas_customizadas.html')
mapa_base